# Storage

In [ ]:
## the path to the `mldl2023` folder in your drive
rootMldl = '<your_drive>/mldl2023'

# Initialization

In [ ]:
# packages 1
import shutil
import os

In [ ]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# copying the idda data into local storage
print('Copying IDDA data ...')
shutil.copytree(rootMldl+'/data/idda', '/content/data/idda', dirs_exist_ok=True)

In [ ]:
# packages 2
import gc
import numpy as np
import tqdm.notebook as tqdm
from PIL import Image
import json

# Different Things

In [ ]:
# params class
class Params:
    def __init__(self, **args):
        for key, value in args.items():
            setattr(self, key, value)

In [ ]:
# style extractor, returns the amplitude of the FFT of each channel of an image
def get_style(image):
    img = np.array(image)
    # fourier transform need to happen across the width and height, not the channels, which are at axis = 2
    # image shape = (1080, 1920, 3)
    imgfft = np.fft.fftshift(np.fft.fft2(img, axes=(0, 1)), axes=(0, 1))
    # get only the amplitude of the FT
    imgfftamplitude = np.abs(imgfft)
    return imgfftamplitude

# Main

In [ ]:
# params
## the intitial parameters
params = Params(
    rootIdda = '/content/data/idda',
    rootStyles = rootMldl+'/data/idda/styles',
)

In [ ]:
# clients names and respective images
with open(params.rootIdda+'/train.json') as file:
    clientsInfo = json.load(file)
    file.close

# making the storage folder
os.makedirs(params.rootStyles, exist_ok=True)

# extracting styles
pbar = tqdm.tqdm(
    total = sum([len(fileNames) for clientName, fileNames in clientsInfo.items()]),
    desc = 'extracting styles'
)
for clientName, fileNames in clientsInfo.items():
    gc.collect()

    style = 0.0

    # calculate the average style for each client and store in separate files
    # where each file is the average style of a client
    for fileName_index, fileName in enumerate(fileNames):
        buffer = Image.open(params.rootIdda+'/images/'+fileName+'.jpg').convert('RGB')
        style = (style*fileName_index + get_style(buffer))/(fileName_index + 1)
        pbar.update(1)

    with open(params.rootStyles+'/'+clientName+'.json', 'w') as file:
        json.dump({clientName:style.tolist()}, file)

pbar.close()